In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import base64
import pandas as pd

# Import the CRUD Python module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Update with your username and password
username = "aacuser"
password = "newyork26"

# Connect to database via CRUD Module
db = AnimalShelter()

# Fetch all data and prepare DataFrame
df = pd.DataFrame.from_records(db.read({}))
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)  # Drop the '_id' column

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Encode the Grazioso Salvare logo
image_filename = 'Grazioso Salvare Logo.png'  # Ensure this file is in the same directory
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# App Layout
app.layout = html.Div([
    # Header Section
    html.Center([
        html.H1("Nathalie Morales CS-340 Dashboard"),
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={'height': '100px', 'margin': 'auto', 'display': 'block'}
        ),
        html.P("Created by: Nathalie Morales | Date: 2024-12-18",
               style={'textAlign': 'center', 'fontWeight': 'bold'})
    ]),
    html.Hr(),

    # Filter Options
    html.Div([
        html.Label("Select Rescue Type:", style={'fontWeight': 'bold'}),
        dcc.RadioItems(
            id='filter-options',
            options=[
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain Rescue', 'value': 'Mountain'},
                {'label': 'Disaster Rescue', 'value': 'Disaster'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',  # Default option
            style={'textAlign': 'center'}
        )
    ], style={'padding': '10px', 'border': '1px solid black', 'margin': '20px'}),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        style_table={'overflowX': 'auto'},
        page_size=10
    ),
    html.Br(),
    html.Hr(),

    # Charts Section: Geolocation and Pie Chart Side-by-Side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(dcc.Graph(id='pie-chart'), className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Update DataTable Based on Filter Selection
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-options', 'value')]
)
def update_table(filter_type):
    filters = {
        "Water": {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                  "sex_upon_outcome": "Intact Female",
                  "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}},
        "Mountain": {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                     "sex_upon_outcome": "Intact Male",
                     "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}},
        "Disaster": {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                     "sex_upon_outcome": "Intact Male",
                     "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
    }

    if filter_type == "Reset":
        filtered_data = db.read({})
    else:
        filtered_data = db.read(filters[filter_type])

    return pd.DataFrame(filtered_data).drop(columns=['_id']).to_dict('records')

# Update Pie Chart Based on Table Data
@app.callback(
    Output('pie-chart', "figure"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_pie_chart(viewData):
    if viewData is None:
        return px.pie(title="No Data Available")
    df_filtered = pd.DataFrame(viewData)
    return px.pie(df_filtered, names='breed', title="Breed Distribution")

# Update Map Based on Table Row Selection
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):    
    if not viewData:
        return html.P("No data to display.")
    dff = pd.DataFrame.from_dict(viewData)
    if index is None or not index:
        row = 0
    else: 
        row = index[0]

    return dl.Map(
        style={'width': '1000px', 'height': '500px'},
        center=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']],
        zoom=10,
        children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(
                position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']],
                children=[
                    dl.Tooltip(dff.iloc[row]['breed']),
                    dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[row]['name'])])
                ]
            )
        ]
    )

# Run Server
app.run_server(debug=True)


Dash app running on http://127.0.0.1:26633/
